# Part 2 preparing and analysing Tokyo data #

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files
#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


## Lets get the Tokyo's data from the csv file ##

In [2]:
toky_df = pd.read_csv('Tokyo_data.csv')
tokyo_df = toky_df.rename(columns={'Neghborhoods':'Neighbourhood'})
tokyo_df.head()

,Neighbourhood,Latitude,Longitude
0,Akihabara,35.7023,139.7745
1,Asakusa,35.7186,139.7959
2,Daikanyama,35.6505,139.7042
3,Denenchofu,35.5939,139.6682
4,Ebisu,35.6461,139.7154


##### Now visualizat Tokyo's neighbourhoods.

In [3]:
address = 'Tokyo'

geolocator = Nominatim(user_agent="ont_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Tokyo are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Tokyo are 35.6828387, 139.7594549.


##### Now visualizat Toronto's neighbourhoods.

In [4]:
# create map of Toronto using latitude and longitude values
map_tokyo = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(tokyo_df['Latitude'], tokyo_df['Longitude'], tokyo_df['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_tokyo)  
    
map_tokyo

In [5]:
tokyo_df.loc[0, 'Neighbourhood']

'Akihabara'

##### Get the Lat and Long of first neighbourhood

In [6]:
neighborhood_latitude = tokyo_df.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = tokyo_df.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = tokyo_df.loc[0, 'Neighbourhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Akihabara are 35.7023, 139.7745.


In [7]:
CLIENT_ID = '5GPUKSCMRUQEQIEZ2OXGPL4Y14UV1WNBKLCEEON3HHHYEKVD' # your Foursquare ID
CLIENT_SECRET = '04YUMWDAHN5BYQR1VYPLMHBHOC0NQEGKAOQKUPFOEZTS03KO' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 5GPUKSCMRUQEQIEZ2OXGPL4Y14UV1WNBKLCEEON3HHHYEKVD
CLIENT_SECRET:04YUMWDAHN5BYQR1VYPLMHBHOC0NQEGKAOQKUPFOEZTS03KO


In [8]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)

###### Send the http get request and ge the result from API contrcat as json file. 

In [9]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5ca472b09fb6b713f916c4e0'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Akihabara',
  'headerFullLocation': 'Akihabara, Tokyo',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 229,
  'suggestedBounds': {'ne': {'lat': 35.7068000045, 'lng': 139.78003112074953},
   'sw': {'lat': 35.6977999955, 'lng': 139.76896887925045}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '53e43bf5498e35f10c4a8bac',
       'name': 'Akiba Fukurou (アキバフクロウ)',
       'location': {'address': '神田練塀町67',
        'lat': 35.70071628709249,
        'lng': 139.77475334149858,
        'labeledLatLngs': [{'label': 'display',
          'lat': 35.70071628709249,

In [10]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

###### Change the json file into pandas dataframe

In [11]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Akiba Fukurou (アキバフクロウ),Pet Café,35.700716,139.774753
1,MOGRA,Nightclub,35.701987,139.775072
2,aitendo,Hobby Shop,35.703707,139.773503
3,Fruit de saison (フルーフ・デゥ・セゾン),Café,35.702276,139.772864
4,e☆イヤホン 秋葉原店,Electronics Store,35.702048,139.772191


In [12]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

100 venues were returned by Foursquare.


## Exporing neighbourhoods in Tokyo
A function that looks at all neighbourhoods in Tokyo and other cities as well

In [13]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [14]:
tokyo_venues = getNearbyVenues(names=tokyo_df['Neighbourhood'],
                                   latitudes=tokyo_df['Latitude'],
                                   longitudes=tokyo_df['Longitude']
                                  )

Akihabara
Asakusa
Daikanyama
Denenchofu
Ebisu
Gaienmae
Ginza
Gotanda
Harajuku
Hibiya
Hiroo
Ikebukuro
Jimbocho
Jiyugaoka
Kagurazaka
Kasumigaseki
Kichijoji
Koenji
Kyobashi
Marunouchi
Meguro
Nagatacho
Nakameguro
Nakano
Nippori
Ochanomizu
Odaiba
Omotesando
Sangenjaya
Shibuya
Shimbashi
Shimokitazawa
Shinagawa
Shinjuku
Shiodome
Sugamo
Takadanobaba
Takebashi
Toranomon
Tsukiji
Ueno
Yoyogi
Yurakucho


In [15]:
print(tokyo_venues.shape)
tokyo_venues.head()

(3329, 7)


,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Akihabara,35.7023,139.7745,Akiba Fukurou (アキバフクロウ),35.700716,139.774753,Pet Café
1,Akihabara,35.7023,139.7745,MOGRA,35.701987,139.775072,Nightclub
2,Akihabara,35.7023,139.7745,aitendo,35.703707,139.773503,Hobby Shop
3,Akihabara,35.7023,139.7745,Fruit de saison (フルーフ・デゥ・セゾン),35.702276,139.772864,Café
4,Akihabara,35.7023,139.7745,e☆イヤホン 秋葉原店,35.702048,139.772191,Electronics Store


#### Here are the unmbers of venues in Tokyo's neighbourhoods

In [16]:
tokyo_venues.groupby('Neighbourhood').count()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
Akihabara,100,100,100,100,100,100
Asakusa,78,78,78,78,78,78
Daikanyama,100,100,100,100,100,100
Denenchofu,39,39,39,39,39,39
Ebisu,100,100,100,100,100,100
Gaienmae,52,52,52,52,52,52
Ginza,100,100,100,100,100,100
Gotanda,100,100,100,100,100,100
Harajuku,100,100,100,100,100,100


In [17]:
print('There are {} uniques categories.'.format(len(tokyo_venues['Venue Category'].unique())))

There are 264 uniques categories.


#### Add Neighbourhood colum as first colum into the dataframe for more analysis

In [18]:
# one hot encoding
tokyo_onehot = pd.get_dummies(tokyo_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
tokyo_onehot['Neighbourhood'] = tokyo_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [tokyo_onehot.columns[-1]] + list(tokyo_onehot.columns[:-1])
tokyo_onehot = tokyo_onehot[fixed_columns]

tokyo_onehot.head()

,Neighbourhood,ATM,Accessories Store,Afghan Restaurant,American Restaurant,Antique Shop,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,BBQ Joint,Bagel Shop,Bakery,Bar,Baseball Stadium,Bath House,Bed & Breakfast,Beer Bar,Beer Garden,Bike Rental / Bike Share,Bike Shop,Bistro,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Buddhist Temple,Buffet,Burger Joint,Bus Stop,Business Center,Butcher,Café,Camera Store,Candy Store,Castle,Cemetery,Cheese Shop,Chinese Restaurant,Chocolate Shop,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,Comedy Club,Comic Shop,Concert Hall,Confucian Temple,Convenience Store,Cosmetics Shop,Costume Shop,Creperie,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Donburi Restaurant,Dongbei Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Event Space,Fabric Shop,Farmers Market,Fast Food Restaurant,Fish Market,Flower Shop,Food & Drink Shop,Food Court,Forest,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Gastropub,German Restaurant,Gift Shop,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Halal Restaurant,Hawaiian Restaurant,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hot Dog Joint,Hot Spring,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Intersection,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kaiseki Restaurant,Karaoke Bar,Karaoke Box,Kebab Restaurant,Kids Store,Korean Restaurant,Kushikatsu Restaurant,Lake,Library,Light Rail Station,Liquor Store,Lounge,Luggage Store,Market,Martial Arts Dojo,Massage Studio,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Molecular Gastronomy Restaurant,Mongolian Restaurant,Motorcycle Shop,Mountain,Movie Theater,Multiplex,Museum,Music Venue,Nabe Restaurant,Nightclub,Noodle House,North Indian Restaurant,Okonomiyaki Restaurant,Optical Shop,Outdoor Event Space,Outdoor Sculpture,Outdoor Supply Store,Outlet Store,Paintball Field,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Peruvian Restaurant,Pet Café,Pet Store,Pharmacy,Pie Shop,Pizza Place,Planetarium,Platform,Playground,Plaza,Pub,Racetrack,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Reservoir,Restaurant,Road,Rock Club,Roof Deck,Rugby Stadium,Russian Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,Science Museum,Seafood Restaurant,Shabu-Shabu Restaurant,Shanghai Restaurant,Shoe Store,Shop & Service,Shopping Mall,Shrine,Skating Rink,Smoke Shop,Snack Place,Soba Restaurant,Soup Place,South Indian Restaurant,Souvenir Shop,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Sri Lankan Restaurant,Stables,Stationery Store,Steakhouse,Sukiyaki Restaurant,Supermarket,Sushi Restaurant,Szechuan Restaurant,Taiwanese Restaurant,Takoyaki Place,Tapas Restaurant,Tea Room,Temple,Tempura Restaurant,Tennis Court,Tennis Stadium,Thai Restaurant,Theater,Theme Park,Theme Restaurant,Thrift / Vintage Store,Tonkatsu Restaurant,Tourist Information Center,Toy / Game Store,Trail,Train Station,Tram Station,Tree,Tunnel,Turkish Restaurant,Udon Restaurant,Unagi Restaurant,Used Bookstore,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wagashi Place,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Xinjiang Restaurant,Yakitori Restaurant,Yoga Studio,Yoshoku Restaurant
0,Akihabara,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

#### Check the new dataframe size

In [19]:
tokyo_onehot.shape

(3329, 265)

##### Grouping row by neighbourhoods and by taking the mean of the frequency of occurrence of each category

In [20]:
tokyo_grouped = tokyo_onehot.groupby('Neighbourhood').mean().reset_index()
tokyo_grouped

,Neighbourhood,ATM,Accessories Store,Afghan Restaurant,American Restaurant,Antique Shop,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,BBQ Joint,Bagel Shop,Bakery,Bar,Baseball Stadium,Bath House,Bed & Breakfast,Beer Bar,Beer Garden,Bike Rental / Bike Share,Bike Shop,Bistro,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Buddhist Temple,Buffet,Burger Joint,Bus Stop,Business Center,Butcher,Café,Camera Store,Candy Store,Castle,Cemetery,Cheese Shop,Chinese Restaurant,Chocolate Shop,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,Comedy Club,Comic Shop,Concert Hall,Confucian Temple,Convenience Store,Cosmetics Shop,Costume Shop,Creperie,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Donburi Restaurant,Dongbei Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Event Space,Fabric Shop,Farmers Market,Fast Food Restaurant,Fish Market,Flower Shop,Food & Drink Shop,Food Court,Forest,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Gastropub,German Restaurant,Gift Shop,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Halal Restaurant,Hawaiian Restaurant,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hot Dog Joint,Hot Spring,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Intersection,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kaiseki Restaurant,Karaoke Bar,Karaoke Box,Kebab Restaurant,Kids Store,Korean Restaurant,Kushikatsu Restaurant,Lake,Library,Light Rail Station,Liquor Store,Lounge,Luggage Store,Market,Martial Arts Dojo,Massage Studio,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Molecular Gastronomy Restaurant,Mongolian Restaurant,Motorcycle Shop,Mountain,Movie Theater,Multiplex,Museum,Music Venue,Nabe Restaurant,Nightclub,Noodle House,North Indian Restaurant,Okonomiyaki Restaurant,Optical Shop,Outdoor Event Space,Outdoor Sculpture,Outdoor Supply Store,Outlet Store,Paintball Field,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Peruvian Restaurant,Pet Café,Pet Store,Pharmacy,Pie Shop,Pizza Place,Planetarium,Platform,Playground,Plaza,Pub,Racetrack,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Reservoir,Restaurant,Road,Rock Club,Roof Deck,Rugby Stadium,Russian Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,Science Museum,Seafood Restaurant,Shabu-Shabu Restaurant,Shanghai Restaurant,Shoe Store,Shop & Service,Shopping Mall,Shrine,Skating Rink,Smoke Shop,Snack Place,Soba Restaurant,Soup Place,South Indian Restaurant,Souvenir Shop,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Sri Lankan Restaurant,Stables,Stationery Store,Steakhouse,Sukiyaki Restaurant,Supermarket,Sushi Restaurant,Szechuan Restaurant,Taiwanese Restaurant,Takoyaki Place,Tapas Restaurant,Tea Room,Temple,Tempura Restaurant,Tennis Court,Tennis Stadium,Thai Restaurant,Theater,Theme Park,Theme Restaurant,Thrift / Vintage Store,Tonkatsu Restaurant,Tourist Information Center,Toy / Game Store,Trail,Train Station,Tram Station,Tree,Tunnel,Turkish Restaurant,Udon Restaurant,Unagi Restaurant,Used Bookstore,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wagashi Place,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Xinjiang Restaurant,Yakitori Restaurant,Yoga Studio,Yoshoku Restaurant
0,Akihabara,0.00,0.00,0.00,0.000000,0.00000,0.020000,0.010000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.030000,0.000000,0.000000,0.020000,0.000000,0.000000,0.010000,0.000000,0.00,0.000000,0.000000,0.01,0.000000,0.000000,0.00

##### Now checking the size of dataframe

In [21]:
tokyo_grouped.shape

(43, 265)

#### Each neighbourhood with the top 5 most common venues

In [22]:
num_top_venues = 5

for hood in tokyo_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = tokyo_grouped[tokyo_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Akihabara----
               venue  freq
0  Electronics Store  0.08
1   Ramen Restaurant  0.07
2         Hobby Shop  0.07
3               Café  0.07
4           Sake Bar  0.05


----Asakusa----
                 venue  freq
0    Convenience Store  0.12
1  Japanese Restaurant  0.06
2     Ramen Restaurant  0.06
3         Dessert Shop  0.05
4               Hostel  0.04


----Daikanyama----
                 venue  freq
0                 Café  0.09
1  Japanese Restaurant  0.08
2   Italian Restaurant  0.05
3          Coffee Shop  0.05
4               Bakery  0.04


----Denenchofu----
                venue  freq
0       Train Station  0.13
1                Park  0.10
2  Italian Restaurant  0.08
3            Platform  0.08
4       Historic Site  0.05


----Ebisu----
                 venue  freq
0   Italian Restaurant  0.09
1  Japanese Restaurant  0.08
2     Ramen Restaurant  0.06
3            BBQ Joint  0.05
4                  Bar  0.04


----Gaienmae----
                    venue  freq
0  

#### The function below will sort venues in descending order

In [23]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

#### Create new dataframe with top 10 venues

In [24]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighbourhoods_venues_sorted = pd.DataFrame(columns=columns)
neighbourhoods_venues_sorted['Neighbourhood'] = tokyo_grouped['Neighbourhood']

for ind in np.arange(tokyo_grouped.shape[0]):
    neighbourhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(tokyo_grouped.iloc[ind, :], num_top_venues)

neighbourhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Akihabara,Electronics Store,Café,Hobby Shop,Ramen Restaurant,Sake Bar,Chinese Restaurant,Donburi Restaurant,Soba Restaurant,BBQ Joint,Yoshoku Restaurant
1,Asakusa,Convenience Store,Japanese Restaurant,Ramen Restaurant,Dessert Shop,Hostel,Wagashi Place,Café,BBQ Joint,Hotel,Bakery
2,Daikanyama,Café,Japanese Restaurant,Coffee Shop,Italian Restaurant,Bakery,Boutique,Nightclub,Sandwich Place,Burger Joint,BBQ Joint
3,Denenchofu,Train Station,Park,Platform,Italian Restaurant,Historic Site,Convenience Store,Coffee Shop,Café,Grocery Store,Sushi Restaurant
4,Ebisu,Italian Restaurant,Japanese Restaurant,Ramen Restaurant,BBQ Joint,Bar,Café,Chinese Restaurant,Soba Restaurant,French Restaurant,Pizza Place


#### Cluster neighbourhoods using k-means to 5 clusters

In [25]:
# set number of clusters
kclusters = 5

tokyo_grouped = tokyo_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(tokyo_grouped)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 3, 2, 3, 3, 3, 3, 0, 2, 3], dtype=int32)

#### Add labels to the clusters

In [26]:
# add clustering labels
neighbourhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

tokyo_merged = tokyo_df

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
tokyo_merged = tokyo_merged.join(neighbourhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

tokyo_merged.head() # check the last columns!

,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Akihabara,35.7023,139.7745,0,Electronics Store,Café,Hobby Shop,Ramen Restaurant,Sake Bar,Chinese Restaurant,Donburi Restaurant,Soba Restaurant,BBQ Joint,Yoshoku Restaurant
1,Asakusa,35.7186,139.7959,3,Convenience Store,Japanese Restaurant,Ramen Restaurant,Dessert Shop,Hostel,Wagashi Place,Café,BBQ Joint,Hotel,Bakery
2,Daikanyama,35.6505,139.7042,2,Café,Japanese Restaurant,Coffee Shop,Italian Restaurant,Bakery,Boutique,Nightclub,Sandwich Place,Burger Joint,BBQ Joint
3,Denenchofu,35.5939,139.6682,3,Train Station,Park,Platform,Italian Restaurant,Historic Site,Convenience Store,Coffee Shop,Café,Grocery Store,Sushi Restaurant
4,Ebisu,35.6461,139.7154,3,Italian Restaurant,Japanese Restaurant,Ramen Restaurant,BBQ Joint,Bar,Café,Chinese Restaurant,Soba Restaurant,French Restaurant,Pizza Place


##### Finally, let's visualize the resulting clusters

In [27]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(tokyo_merged['Latitude'], tokyo_merged['Longitude'], tokyo_merged['Neighbourhood'], tokyo_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Examin the clusters
#### Cluster 1

In [28]:
tokyo_merged.loc[tokyo_merged['Cluster Labels'] == 0, tokyo_merged.columns[[0] + list(range(3, tokyo_merged.shape[1]))]]

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Akihabara,0,Electronics Store,Café,Hobby Shop,Ramen Restaurant,Sake Bar,Chinese Restaurant,Donburi Restaurant,Soba Restaurant,BBQ Joint,Yoshoku Restaurant
7,Gotanda,0,Sake Bar,BBQ Joint,Bar,Japanese Restaurant,Chinese Restaurant,Ramen Restaurant,Steakhouse,Seafood Restaurant,Italian Restaurant,Sushi Restaurant
11,Ikebukuro,0,Sake Bar,Ramen Restaurant,Chinese Restaurant,BBQ Joint,Yoshoku Restaurant,Hotel,Park,Japanese Restaurant,Convenience Store,Soba Restaurant
17,Koenji,0,Sake Bar,Café,Bar,Convenience Store,BBQ Joint,Indian Restaurant,Italian Restaurant,Ramen Restaurant,Dumpling Restaurant,Yakitori Restaurant
23,Nakano,0,Ramen Restaurant,Sake Bar,Japanese Restaurant,Italian Restaurant,BBQ Joint,Café,Japanese Curry Restaurant,Bar,Seafood Restaurant,Ice Cream Shop
24,Nippori,0,Sake Bar,Café,Convenience Store,Ramen Restaurant,Bed & Breakfast,Japanese Restaurant,Ice Cream Shop,Climbing Gym,Shopping Mall,Hotel
30,Shimbashi,0,Sake Bar,Japanese Restaurant,BBQ Joint,Ramen Restaurant,Seafood Restaurant,Chinese Restaurant,Tonkatsu Restaurant,Udon Restaurant,Japanese Curry Restaurant,Bed & Breakfast
31,Shimokitazawa,0,Ramen Restaurant,Japanese Curry Restaurant,Coffee Shop,Café,Sake Bar,Rock Club,Music Venue,Bar,Seafood Restaurant,Record Shop
33,Shinjuku,0,Bar,Sake Bar,Ramen Restaurant,Japanese Restaurant,Multiplex,Yakitori Restaurant,BBQ Joint,Pub,Thai Restaurant,Tonkatsu Restaurant
34,Shiodome,0,Sake Bar,Japanese Restaurant,BBQ Joint,Chinese Restaurant,Ramen Restaurant,Hotel,Seafood Restaurant,Udon Restaurant,Coffee Shop,Bistro


In [29]:
tokyo_merged.loc[tokyo_merged['Cluster Labels'] == 1, tokyo_merged.columns[[0] + list(range(3, tokyo_merged.shape[1]))]]

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
15,Kasumigaseki,1,Convenience Store,Historic Site,Restaurant,Café,Intersection,Park,Museum,Bridge,Taiwanese Restaurant,Bus Stop
21,Nagatacho,1,Convenience Store,Coffee Shop,Hotel,Japanese Restaurant,Lake,Ramen Restaurant,Bus Stop,Sports Club,Chinese Restaurant,Pizza Place
26,Odaiba,1,Convenience Store,Intersection,Japanese Restaurant,Plaza,Planetarium,Gift Shop,Outlet Store,Steakhouse,Chinese Restaurant,Frozen Yogurt Shop
37,Takebashi,1,Coffee Shop,Convenience Store,Historic Site,Italian Restaurant,Chinese Restaurant,Japanese Restaurant,Bridge,Tonkatsu Restaurant,Museum,Garden
41,Yoyogi,1,Convenience Store,Indian Restaurant,Café,Hotel,Bar,Ramen Restaurant,Chinese Restaurant,Coffee Shop,Japanese Restaurant,French Restaurant


In [30]:
tokyo_merged.loc[tokyo_merged['Cluster Labels'] == 2, tokyo_merged.columns[[0] + list(range(3, tokyo_merged.shape[1]))]]

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Daikanyama,2,Café,Japanese Restaurant,Coffee Shop,Italian Restaurant,Bakery,Boutique,Nightclub,Sandwich Place,Burger Joint,BBQ Joint
8,Harajuku,2,Café,Clothing Store,Coffee Shop,Shoe Store,Toy / Game Store,Luggage Store,Boutique,Thai Restaurant,Japanese Restaurant,Record Shop
10,Hiroo,2,Café,Convenience Store,French Restaurant,Coffee Shop,Japanese Restaurant,Chinese Restaurant,Ramen Restaurant,Bakery,Yoshoku Restaurant,Hawaiian Restaurant
12,Jimbocho,2,Café,Ramen Restaurant,Japanese Curry Restaurant,Chinese Restaurant,Bookstore,Yoshoku Restaurant,Indian Restaurant,Sake Bar,Tempura Restaurant,Thai Restaurant
13,Jiyugaoka,2,Café,Dessert Shop,Italian Restaurant,Bakery,Furniture / Home Store,Convenience Store,Coffee Shop,Ramen Restaurant,Recording Studio,Thai Restaurant
25,Ochanomizu,2,Café,Electronics Store,Ramen Restaurant,Japanese Curry Restaurant,Hobby Shop,Soba Restaurant,Japanese Restaurant,Record Shop,Donburi Restaurant,Chinese Restaurant
29,Shibuya,2,Café,Japanese Restaurant,Coffee Shop,Nightclub,French Restaurant,Tonkatsu Restaurant,Clothing Store,Art Gallery,Italian Restaurant,Bakery


In [31]:
tokyo_merged.loc[tokyo_merged['Cluster Labels'] == 3, tokyo_merged.columns[[0] + list(range(3, tokyo_merged.shape[1]))]]

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Asakusa,3,Convenience Store,Japanese Restaurant,Ramen Restaurant,Dessert Shop,Hostel,Wagashi Place,Café,BBQ Joint,Hotel,Bakery
3,Denenchofu,3,Train Station,Park,Platform,Italian Restaurant,Historic Site,Convenience Store,Coffee Shop,Café,Grocery Store,Sushi Restaurant
4,Ebisu,3,Italian Restaurant,Japanese Restaurant,Ramen Restaurant,BBQ Joint,Bar,Café,Chinese Restaurant,Soba Restaurant,French Restaurant,Pizza Place
5,Gaienmae,3,Italian Restaurant,Baseball Stadium,Furniture / Home Store,Café,Hotel,Coffee Shop,Chinese Restaurant,Rock Club,French Restaurant,Japanese Curry Restaurant
6,Ginza,3,Coffee Shop,Ramen Restaurant,Dessert Shop,Boutique,French Restaurant,Japanese Restaurant,Chocolate Shop,Shabu-Shabu Restaurant,Clothing Store,Bar
9,Hibiya,3,Café,Sake Bar,Bar,Italian Restaurant,French Restaurant,Theater,Sushi Restaurant,Japanese Restaurant,Ramen Restaurant,Hotel
14,Kagurazaka,3,Italian Restaurant,Japanese Restaurant,Sake Bar,French Restaurant,Coffee Shop,Ramen Restaurant,Soba Restaurant,Steakhouse,Chinese Restaurant,Café
16,Kichijoji,3,Japanese Restaurant,Café,Ramen Restaurant,Sake Bar,Tea Room,Steakhouse,Italian Restaurant,Yakitori Restaurant,Bookstore,Tonkatsu Restaurant
18,Kyobashi,3,Ramen Restaurant,Italian Restaurant,Café,Japanese Restaurant,Yakitori Restaurant,Dessert Shop,French Restaurant,South Indian Restaurant,BBQ Joint,Gourmet Shop
19,Marunouchi,3,Hobby Shop,Ramen Restaurant,Café,Japanese Restaurant,Gift Shop,South Indian Restaurant,Deli / Bodega,Hotel,Bakery,Train Station


In [32]:
tokyo_merged.loc[tokyo_merged['Cluster Labels'] == 4, tokyo_merged.columns[[0] + list(range(3, tokyo_merged.shape[1]))]]

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
27,Omotesando,4,Playground,Temple,Restaurant,Park,Supermarket,Bike Rental / Bike Share,Convenience Store,Grocery Store,Fabric Shop,Frozen Yogurt Shop
